# Short Breakout Strategy Testing

This notebook tests the **ShortBreakoutStrategy** which implements a contrarian approach to market breakouts.

## Strategy Overview
- **Concept**: Short positions on upward breakouts, betting on price reversion
- **Entry Signal**: Price breaks above recent high by a threshold percentage
- **Exit Signal**: Hold for a specified time period, then close
- **Risk Management**: Limited position size and concurrent position limits

## Strategy Parameters
- **Breakout Threshold**: Minimum percentage above recent high to trigger entry
- **Lookback Period**: Time window to calculate recent high/low
- **Hold Time**: How long to maintain positions before closing
- **Portfolio Percentage**: Percentage of total portfolio value to allocate for position sizing

## Test Components
1. **Data Loading & Visualization**
2. **Single Backtest with Default Parameters**
3. **Performance Analysis & Metrics**
4. **Parameter Sensitivity Testing**
5. **Risk Analysis & Trade Visualization**

Let's test this contrarian strategy on Bitcoin data!

In [15]:
# Setup environment and imports
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Environment setup complete!")
print("Modules loaded successfully")

Environment setup complete!
Modules loaded successfully


In [16]:
# Load Bitcoin data for testing
from backtester.data_loader import DataLoader

# Initialize DataLoader
loader = DataLoader(base_path="../data")

# Load Bank data
print("Loading Bank (BANKUSDT) data...")
btc_df = loader.load_symbol("BANKUSDT", file_type="csv")

print("✅ Data loaded successfully!")
print(f"📊 Data shape: {btc_df.shape}")
print(f"📅 Date range: {btc_df['timestamp'].min()} to {btc_df['timestamp'].max()}")

# Convert timestamp to datetime
btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'])

# Display basic statistics
print(f"\n💰 Price Statistics:")
print(f"   Min price: ${btc_df['low'].min():,.2f}")
print(f"   Max price: ${btc_df['high'].max():,.2f}")
print(f"   Start price: ${btc_df['close'].iloc[0]:,.2f}")
print(f"   End price: ${btc_df['close'].iloc[-1]:,.2f}")
print(f"   Total return (Buy & Hold): {((btc_df['close'].iloc[-1] / btc_df['close'].iloc[0]) - 1) * 100:.2f}%")

# Quick preview
print(f"\n📋 Data Preview:")
display(btc_df.head())

Loading Bank (BANKUSDT) data...
✅ Data loaded successfully!
📊 Data shape: (261439, 6)
📅 Date range: 2025-04-18 20:30:00 to 2025-10-17 09:48:00

💰 Price Statistics:
   Min price: $0.03
   Max price: $0.19
   Start price: $0.05
   End price: $0.14
   Total return (Buy & Hold): 190.20%

📋 Data Preview:
✅ Data loaded successfully!
📊 Data shape: (261439, 6)
📅 Date range: 2025-04-18 20:30:00 to 2025-10-17 09:48:00

💰 Price Statistics:
   Min price: $0.03
   Max price: $0.19
   Start price: $0.05
   End price: $0.14
   Total return (Buy & Hold): 190.20%

📋 Data Preview:


,timestamp,open,high,low,close,volume
0,2025-04-18 20:30:00,0.040000,0.050176,0.040000,0.048235,4936472.0
1,2025-04-18 20:31:00,0.048235,0.052205,0.048045,0.051427,7204774.0
2,2025-04-18 20:32:00,0.051246,0.054277,0.048300,0.049230,20698871.0
3,2025-04-18 20:33:00,0.049467,0.052000,0.049155,0.051869,10414589.0
4,2025-04-18 20:34:00,0.051678,0.053405,0.051102,0.051676,8885868.0


In [17]:
# Run backtest with Short Breakout Strategy
from backtester.engine import BacktesterEngine
from backtester.broker import Broker
from backtester.portfolio import Portfolio
from backtester.Strategies.short_breakout_strategy import ShortBreakoutStrategy

print("🚀 Setting up Short Breakout Strategy backtest...\n")

# Initialize portfolio and broker
initial_cash = 100_000  # $100,000 starting capital
portfolio = Portfolio(initial_cash=initial_cash)
broker = Broker(portfolio, fee_rate=0.001)  # 0.1% trading fee

# Create ShortBreakoutStrategy with updated parameters
symbols = ["BTCUSDT"]
strategy = ShortBreakoutStrategy(
    symbol="BTCUSDT",
    breakout_threshold=0.05,      # 5% breakout threshold
    lookback_period_minutes=10,   # 30-minute lookback
    hold_time_minutes=60,         # Hold positions for 60 minutes
    portfolio_percentage=30.0     # Use 30% of portfolio for position sizing
)

# Initialize backtester engine
engine = BacktesterEngine([strategy], symbols, broker, portfolio)

# Load data into engine
print("📊 Loading data into backtest engine...")
engine.load_data("BTCUSDT", btc_df)
print(f"   Loaded {len(btc_df)} candles for backtesting")

# Initialize strategies
print("🎯 Initializing strategy...")
engine.initialize()

print("\n⚡ Starting backtest execution...")
print("⏱️  This may take a moment for large datasets...")

# Run the backtest
start_time = datetime.now()
engine.run_backtest()
end_time = datetime.now()

execution_time = (end_time - start_time).total_seconds()
print(f"\n✅ Backtest completed in {execution_time:.2f} seconds!")
print(f"📈 Processing rate: {len(btc_df) / execution_time:.0f} candles/second")

🚀 Setting up Short Breakout Strategy backtest...

📊 Loading data into backtest engine...
   Loaded 261439 candles for backtesting
🎯 Initializing strategy...
ShortBreakoutStrategy initialized for BTCUSDT
  Breakout threshold: 5.0%
  Lookback period: 10 minutes
  Hold time: 60 minutes
  Portfolio percentage: 30.0% of total portfolio

⚡ Starting backtest execution...
⏱️  This may take a moment for large datasets...
Breakout detected! Submitting SHORT order for 611583.389395 at $0.05 (breakout: 5.49%)
Fill: SOLD 611583.389395 BTCUSDT at $0.05, fee: $30.0000
Submitting close order: BUY 611583.389395 at $0.05 (held for 60.0 min)
Fill: BOUGHT 611583.389395 BTCUSDT at $0.05, fee: $30.1933
Closed position: filled 611583.389395 at $0.05
Breakout detected! Submitting SHORT order for 544210.597544 at $0.05 (breakout: 6.27%)
Fill: SOLD 544210.597544 BTCUSDT at $0.05, fee: $29.9240
Submitting close order: BUY 544210.597544 at $0.05 (held for 60.0 min)
Fill: BOUGHT 544210.597544 BTCUSDT at $0.05, fee

In [18]:
# Debug: Check the fills to understand the sequence
print("🔍 DETAILED FILL ANALYSIS")
print("=" * 50)

fills = broker.get_fills()
for i, fill in enumerate(fills):
    action = "BOUGHT" if fill.is_buy else "SOLD"
    print(f"Fill {i+1}: {action} {fill.size:.6f} at ${fill.price:.5f}, fee: ${fill.fee:.4f}")

print(f"\n📊 Portfolio Position: {portfolio.get_position('BTCUSDT').size:.6f}")

🔍 DETAILED FILL ANALYSIS
Fill 1: SOLD 611583.389395 at $0.04905, fee: $30.0000
Fill 2: BOUGHT 611583.389395 at $0.04937, fee: $30.1933
Fill 3: SOLD 544210.597544 at $0.05499, fee: $29.9240
Fill 4: BOUGHT 544210.597544 at $0.05059, fee: $27.5322
Fill 5: SOLD 757520.190783 at $0.04043, fee: $30.6243
Fill 6: BOUGHT 757520.190783 at $0.04065, fee: $30.7940
Fill 7: SOLD 418899.356046 at $0.07294, fee: $30.5549
Fill 8: BOUGHT 418899.356046 at $0.06923, fee: $29.0021
Fill 9: SOLD 430237.701893 at $0.07206, fee: $31.0029
Fill 10: BOUGHT 430237.701893 at $0.06788, fee: $29.2054
Fill 11: SOLD 522363.688489 at $0.06035, fee: $31.5241
Fill 12: BOUGHT 522363.688489 at $0.05702, fee: $29.7878
Fill 13: SOLD 479261.260488 at $0.06682, fee: $32.0266
Fill 14: BOUGHT 479261.260488 at $0.07939, fee: $38.0466
Fill 15: SOLD 183083.428577 at $0.16495, fee: $30.1996
Fill 16: BOUGHT 183083.428577 at $0.13764, fee: $25.1996

📊 Portfolio Position: 0.000000


In [19]:
# Comprehensive Results Analysis
print("=" * 60)
print("🎯 SHORT BREAKOUT STRATEGY - BACKTEST RESULTS")
print("=" * 60)

# Portfolio Performance
current_prices = {symbols[0]: btc_df['close'].iloc[-1]}
final_equity = portfolio.equity(current_prices)
total_return = (final_equity - initial_cash) / initial_cash * 100
buy_hold_return = (btc_df['close'].iloc[-1] / btc_df['close'].iloc[0] - 1) * 100

print(f"💰 PORTFOLIO PERFORMANCE")
print(f"   Initial Capital:    ${initial_cash:>10,.2f}")
print(f"   Final Equity:       ${final_equity:>10,.2f}")
print(f"   Total Return:       {total_return:>10.2f}%")
print(f"   Absolute Profit:    ${final_equity - initial_cash:>10,.2f}")

print(f"\n📊 BENCHMARK COMPARISON")
print(f"   Buy & Hold Return:  {buy_hold_return:>10.2f}%")
print(f"   Strategy Return:    {total_return:>10.2f}%")
print(f"   Excess Return:      {total_return - buy_hold_return:>10.2f}%")
if buy_hold_return != 0:
    alpha = total_return - buy_hold_return
    print(f"   Alpha:              {alpha:>10.2f}%")

# Position Analysis
print(f"\n🎯 POSITION ANALYSIS")
print(f"   Cash Remaining:     ${portfolio.cash:>10,.2f}")
for symbol, position in portfolio.positions.items():
    if position.size != 0:
        current_price = btc_df['close'].iloc[-1]
        unrealized_pnl = position.size * (current_price - position.avg_price)
        print(f"   {symbol} Position:      {position.size:>10.6f}")
        print(f"   Average Price:      ${position.avg_price:>10.2f}")
        print(f"   Current Price:      ${current_price:>10.2f}")
        print(f"   Unrealized PnL:     ${unrealized_pnl:>10.2f}")
        print(f"   Realized PnL:       ${position.realized_pnl:>10.2f}")

# Trade Statistics
fills = broker.get_fills()
print(f"\n📈 TRADE STATISTICS")
print(f"   Total Trades:       {len(fills):>10d}")

if fills:
    buy_trades = [f for f in fills if f.is_buy]
    sell_trades = [f for f in fills if not f.is_buy]
    
    print(f"   Buy Orders:         {len(buy_trades):>10d}")
    print(f"   Sell Orders:        {len(sell_trades):>10d}")
    
    total_fees = sum(f.fee for f in fills)
    total_volume = sum(f.price * f.size for f in fills)
    avg_trade_size = np.mean([f.price * f.size for f in fills])
    
    print(f"   Total Fees Paid:    ${total_fees:>10.2f}")
    print(f"   Total Volume:       ${total_volume:>10,.2f}")
    print(f"   Average Trade Size: ${avg_trade_size:>10,.2f}")
    
    # Calculate trade frequency
    time_span = (btc_df['timestamp'].iloc[-1] - btc_df['timestamp'].iloc[0]).total_seconds() / 3600  # hours
    trades_per_hour = len(fills) / time_span
    print(f"   Trading Frequency:  {trades_per_hour:>10.2f} trades/hour")

print(f"\n" + "=" * 60)

🎯 SHORT BREAKOUT STRATEGY - BACKTEST RESULTS
💰 PORTFOLIO PERFORMANCE
   Initial Capital:    $100,000.00
   Final Equity:       $105,609.98
   Total Return:             5.61%
   Absolute Profit:    $  5,609.98

📊 BENCHMARK COMPARISON
   Buy & Hold Return:      190.20%
   Strategy Return:          5.61%
   Excess Return:         -184.59%
   Alpha:                 -184.59%

🎯 POSITION ANALYSIS
   Cash Remaining:     $105,609.98

📈 TRADE STATISTICS
   Total Trades:               16
   Buy Orders:                  8
   Sell Orders:                 8
   Total Fees Paid:    $    485.62
   Total Volume:       $485,617.34
   Average Trade Size: $ 30,351.08
   Trading Frequency:        0.00 trades/hour

